# CS 533 Assignment 5 - Ben Whitehead

## Context

This assignment is designed to develop your ability to reason about classifier accuracy metrics in the context of their social impact.

You will do this by partially replicating the ProPublica analysis of fairness in the COMPAS pre-trial risk assessment tool, reflecting on the process, and replicating simulation studies of tradeoffs in the fairness of machine learning metrics.

**Note:** my data is being loaded from a data/ directory. If you do not have that setup in your notebook, you will need to either change the path the data set is loaded from or put your data in a data directory.


## Setup

I am going to start by getting the infectious disease data into a good working state. The data comes in multiple files, so we'll need to concatenate it, and turn it into both state and county level data. 

## Part 1. Load Data

Download the Raw text files for the main ‘BBC’ data set from http://mlg.ucd.ie/datasets/bbc.html. This will be a Zip file that contains text files with data.

**note:** my data is located in the `./data` directory instead of the root of my project

In [2]:
import pandas as pd
import pandas_ml as pdml
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import addfips



ghg = pd.DataFrame()

for f in os.listdir('data/2018_data_summary_spreadsheets'):
    temp = pd.read_excel('data/2018_data_summary_spreadsheets/'+f, sheet_name=0)
    temp['Year'] = f.split('.')[0].split('_')[2]    
    ghg = pd.concat([temp, ghg])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
ghg.reset_index()
ghg.head()

,Address,Adipic Acid Production,Aluminum Production,Ammonia Manufacturing,Biogenic CO2 emissions (metric tons),CO2 emissions (non-biogenic),Cement Production,City,County,Does the facility employ continuous emissions monitoring?,...,Soda Ash Manufacturing,State,Stationary Combustion,Titanium Dioxide Production,Total reported direct emissions,Underground Coal Mines,Very Short-lived Compounds emissions,Year,Zinc Production,Zip Code
0,3820 SAM RAYBURN HIGHWAY,NaN,NaN,NaN,NaN,NaN,NaN,MELISSA,COLLIN COUNTY,N,...,NaN,TX,NaN,NaN,250497.500,NaN,NaN,2017,NaN,75454
1,4200 S. Hwy 15,NaN,NaN,NaN,NaN,NaN,NaN,Hazard,PERRY COUNTY,N,...,NaN,KY,NaN,NaN,218699.250,218699.25,NaN,2017,NaN,40701
2,1845 S. KY HWY 15,NaN,NaN,NaN,NaN,NaN,NaN,Hazard,PERRY,N,...,NaN,KY,NaN,NaN,70265.000,70265.00,NaN,2017,NaN,41701
3,22845 Highway 33,NaN,NaN,NaN,NaN,9120.1,NaN,McKittrick,NaN,N,...,NaN,CA,9115.416,NaN,9298.916,NaN,NaN,2017,NaN,93251
4,730 3rd Avenue,NaN,NaN,NaN,NaN,52645.5,NaN,BROOKLYN,Kings,N,...,NaN,NY,65.000,NaN,52698.656,NaN,NaN,2017,NaN,11232


In [17]:
af = addfips.AddFIPS()
def get_fips(x):
    if math.isnan(x.County) or math.isnan(x.State):
        return -1
    return af.get_county_fips(x.County, state=x.State)

ghg['FIPS'] = ghg.apply(get_fips, axis=1)

NameError: ("name 'math' is not defined", 'occurred at index 0')

In [32]:
all_data.groupby(['Year','State', 'City'])['Total reported direct emissions'].agg('sum')

Year  State  City             
2010  AK     ANCHORAGE            1.989695e+06
             Akutan               3.693258e+04
             Anchorage            8.823022e+05
             BARROW               4.268854e+04
             BIG LAKE             5.252580e+03
             CLEAR AIR STATION    8.999312e+04
             DUTCH HARBOR         2.805247e+04
             EAGLE RIVER          3.231570e+05
             EIELSON AFB          3.306647e+05
             FAIRBANKS            4.540996e+05
             FORT WAINWRIGHT      3.734709e+05
             Fairbanks            1.374978e+05
             HEALY                2.786180e+05
             KENAI                4.477359e+05
             KOTZEBUE             1.468131e+05
             Kenai                3.151448e+05
             Milne Point Unit     1.822157e+05
             NIKISKI              1.813932e+05
             NORTH POLE           5.647467e+05
             North Slope          1.242964e+06
             Offshore        